In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Model

In [3]:
#model
from src.models.resnets_eff import ResNet18

NUM_CLASSES = 10
model = ResNet18(num_classes=NUM_CLASSES).to(device)

# Criterion

In [4]:
from src.common.common import LOSS_NAME_MAP
from src.models.losses import ClassificationLoss

criterion = ClassificationLoss(LOSS_NAME_MAP['ce']())

# Dataset

In [5]:
from torch.utils.data import DataLoader
from src.data.datasets import get_cifar10


train_dataset, _, test_dataset = get_cifar10('data/')

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

loaders = {
    'train': train_loader,
    'test': test_loader
}

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Optimizer

In [23]:
from src.common.common import OPTIMIZER_NAME_MAP, SCHEDULER_NAME_MAP
from src.common.utils import configure_optimizer

GRAD_ACCUM_STEPS = 1
EPOCHS = 100
T_max = (len(train_loader) // GRAD_ACCUM_STEPS) * EPOCHS

optim = OPTIMIZER_NAME_MAP['adamw'](filter(lambda p: p.requires_grad, model.parameters()), **{'lr': 0.01, 'weight_decay': 0.001})
lr_scheduler = SCHEDULER_NAME_MAP['cosine'](optim, T_max=T_max, eta_min=1e-6)

In [ ]:
%tensorboard --logdir=reports/pretraining_resnet18

# Trainer

In [ ]:
from src.trainer.trainer_classification import TrainerClassification

params_trainer = {
    'model': model,
    'criterion': criterion,
    'loaders': loaders,
    'optim': optim,
    'lr_scheduler': lr_scheduler,
}

trainer = TrainerClassification(**params_trainer)

In [ ]:
from src.common.utils import AttrDict

EXP_NAME = 'pretraining_resnet18'

config = {
    'epoch_start_at': 0,
    'epoch_end_at': EPOCHS,
    'save_multi': T_max // 10,
    'grad_accum_steps': GRAD_ACCUM_STEPS,
    'log_multi': 100,
    'whether_clip': False,
    'clip_value': 2.0,
    'base_path': 'reports',
    'exp_name': EXP_NAME,
    'logger_name': 'tensorboard',
    'logger_config': {'api_token': "07a2cd842a6d792d578f8e6c0978efeb8dcf7638", 'project': 'early_exit', 'hyperparameters': {}},
    'random_seed': 42,
    'device': device

}
config = AttrDict(config)

trainer.run_exp(config)

In [ ]:
from src.common.utils import save_model

In [ ]:
save_model(trainer.model, 'models/resnet18_100_epochs.pth')